# TP2
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 1

### Descrição do Problema

É nos dado um Control Flow Automaton (CFA) que descreve um programa imperativo cujo objetivo é implementar a multiplicação de dois inteiros a,b, fornecidos como "input" e um n, também fornecido como "input", de precisão limitada.
Para alem disso, temos de ter em conta os seguintes aspetos:
- Existe a possibilidade de alguma das operações do programa produzir um erro de “overflow”;
- Os nós do grafo representam ações  que actuam sobre os “inputs” do nó e produzem um “output” com as operações indicadas;
- Os ramos do grafo representam ligações que transferem o “output” de um nodo para o “input” do nodo seguinte. Esta transferência é condicionada pela satisfação da condição associada ao ramo.

### Abordagem do Problema

Para resolver este problema, vamos construir um First Order Transition System (FOTS) usando um BitVector de tamanho n de forma a descrever o comportamento do autómato acima mencionado.

( Talvez seja fixe mencionar a escolha do solver aqui futuramente)

Para além do FOTS, também vamos verificar se $P≡(x∗y+z=a∗b)$ é um invariante do comportamento que estamos a estudar.


## Código Python

variaveis -> x,y,z e o program counter (nº de linhas de código)

x,y,z = a,b,0

ideia 1:
0: while(y!=0):

1: 	if even(y) then x,y,z = 2*x,y/2,z

2:	if odd(y)  then x,y,z = x,y-1,z+x

3: stop

ideia 2:

0: while(y!=0):

 	if even(y) then x,y,z = 2*x,y/2,z

	if odd(y)  then x,y,z = x,y-1,z+x

1: stop


outras ideias que podem ser importantes: na aula T do dia 4 vimos que o stor evita usar o diferente por causa de ter problemas com indicibilidade, devemos também fazer o mesmo, vou cometer o erro aqui em baixo de usar not equals mas talvez temos de trocar depois.não esquecer

In [6]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import random as rn

# vou declarar aqui um exemplo depois passamos para exemplos separados
n = 2 # [0,0] [0,1] [1,0] [1,1]
a = 2
b = 2

 $$\mathsf{trans}(x,y,z,pc,x',y',z',pc')\;\equiv\;$$
 $$\left\{\begin{array}{lr} (y=0)\land (pc=0) \land (x'=x)\land (y'=y) \land (c'=c) \land (pc'=3) & \lor \\ (a > b)\land(c=0)\land (a'+b=a) \land (b'=b)\land (c'=0) & \lor \\ (a < b)\land (c=0) \land (b'+a = b)\land (a'=a)\land (c'=0) & \lor \\ (c=1)\land(c'=1)\land (a'=a)\land (b'=b) & \end{array}\right.$$


In [1]:
# Ideias à toa, não faz muito sentido o que está escrito aqui a baixo

def geraBV(valor,n):       
    return BV(valor,n)

def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),types.BVType(n))
    state['y'] = Symbol('y'+str(i),types.BVType(n))
    state['z'] = Symbol('z'+str(i),types.BVType(n))
    return state

def init(state):
    return And(Equals(state['pc'],Int(0)),Equals(BVComp(state['z'], geraBV(0,n)), Equals(BVComp(state['x'], geraBV(a,n)), Equals(BVComp(state['y'], geraBV(b,n)), Int(1)))

def trans(curr,prox):
    tToStop = And(Equals(curr['y'],Int(0)),Equals(curr['pc'],Int(0)),Equals(prox['x'],curr['x']), Equals(prox['y'],curr['y']), Equals(prox['c'],curr['c']), Equals(prox['pc'],Int(1)))
    tEven   = And(Equals(curr['pc'],Int(0)),Equals(BVExtract(curr['y'],n-1),Int(0)), Equals(BVExtract(curr['y'],0,0),Int(0)), Equals(prox['pc'],Int(0)), Equals(prox['z'],curr['z']), Equals(prox['x'], BVLShl(curr['x'],geraBV(2,n))), Equals(prox['y'], BVLShr(curr['y'],geraBV(2,n))))
    tError  = And(Equals(curr['pc'],Int(0)),Equals(BVExtract(curr['y'],n-1),Int(0)), Equals(BVExtract(curr['y'],0,0),Int(1)), Equals(prox['pc'],Int(2)), Equals(prox['z'],curr['z']), Equals(prox['x'], curr['x']), Equals(prox['y'],curr['y']))
    tOdd    = And(Equals(curr['pc'],Int(0)),Equals(BVExtract(curr['y'],n-1),Int(1)), Equals(prox['pc'],Int(0)), Equals(prox['z'],BVAdd(curr['z'],curr['x'])), Equals(prox['x'], curr['x']), Equals(prox['y'],BVAdd(curr['y'],geraBV(1,n))))
    tEnd    = And(Equals(curr['pc'],Int(1)), Equals(prox['x'],curr['x']),Equals(prox['y'],curr['y']),Equals(prox['z'],curr['z']),Equals(prox['pc'],Int(1)))
    return Or(tToStop,tEven,tOdd,tEnd)



SyntaxError: '(' was never closed (1681994378.py, line 15)